In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")



In [2]:
# Reading all data for different data sets

train_df = pd.read_csv('Description_Day_Quantity.csv')

train_df_customers = pd.read_csv('Customer_Description_Day_Quantity.csv')

train_df_NoDesc = pd.read_csv('Customer_Day_Quantity.csv')

test_data1 = pd.read_csv('Test_df_use.csv')
test_df_org = test_data1.copy()

train_df_DescQuantDays = train_df.copy()


In [3]:
test_data1.head()

#var_list = [\
#       'Quantity_Median', 'Quantity_Max', 'Quantity_Min', 'Quantity_StDev'\
#           ]

#var_list = ['Price_MeanDesc', 'Price_MedianDesc',
#       'Price_MaxDesc', 'Price_MinDesc', 'Price_StDevDesc',
#       'Price_Mean_Customer', 'Price_StDev_Customer', 'Quantity_Mean',
#       'Quantity_Median', 'Quantity_Max', 'Quantity_Min', 'Quantity_StDev',
#       'Price_Mode1']

var_list = []

In [4]:
#Divide test dataset in two parts with Description in train and without Description in train
description_list_train = list(train_df_DescQuantDays['Description'].drop_duplicates())
description_list_test = list(test_data1['Description'].drop_duplicates())

print(len(description_list_train))
print(len(description_list_test))

description_list_test_only = (set(description_list_test) - set(description_list_train))

print(len(description_list_test_only))

test_data_unique_desc = test_data1[test_data1['Description'].isin(description_list_test_only)]
test_data_rep_desc = test_data1[~test_data1['Description'].isin(description_list_test_only)]


3789
3563
90


In [5]:
# Resetting Index

test_data_unique_desc.index = range(len(test_data_unique_desc))
test_data_rep_desc.index = range(len(test_data_rep_desc))

In [6]:
train_df_DescQuantDays.head()
test_data_rep_desc.head()

,CustomerID,Description,Quantity,Country35,days_since_first_sold,Price_Mode_Sum,Price_MeanDesc,Price_MedianDesc,Price_MaxDesc,Price_MinDesc,Price_StDevDesc,Price_Mean_Customer,Price_StDev_Customer,Quantity_Mean,Quantity_Median,Quantity_Max,Quantity_Min,Quantity_StDev,Price_Mode1
0,16161,1079,3,1,72,3.30,1.650000,1.65,1.65,1.65,0.000000,3.364909,3.475940,7.684211,6.0,48,1,9.389774,1.65
1,17341,3457,1,1,6,2.08,1.193421,1.25,2.46,0.83,0.272806,2.082536,1.563595,5.527273,3.0,24,1,5.776243,1.25
2,15158,694,36,1,328,7.70,3.871658,4.25,7.46,2.95,0.489634,1.820000,1.376554,5.258389,4.0,36,1,7.243505,4.25
3,16033,3473,2,1,206,3.10,1.642658,1.65,3.29,1.25,0.261951,3.418571,2.832470,11.314779,12.0,192,1,15.310779,1.65
4,15351,871,1,1,339,23.70,12.654669,12.75,24.96,8.12,0.863610,2.833780,2.262824,3.704104,2.0,160,1,10.071870,12.75


# Models

In [7]:
# Multiple Regression

def MultipleLinearRegression_Regressor(train_df):
    # Importing the dataset
    dataset = train_df.copy()
    #print(len(dataset))

    X_train = dataset.iloc[:, :-1].values
    y_train = dataset.iloc[:, -1].values

    # Training the Multiple Linear Regression model on the Training set
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)

     # Predicting the Test set results
    y_pred = regressor.predict(X_train)
    #print(np.concatenate((y_pred.reshape(len(y_pred),1), y_train.reshape(len(y_train),1)),1))

    # Evaluating the Model Performance
    rscore = r2_score(y_train, y_pred)
    rmse_score = np.sqrt(metrics.mean_squared_error(y_train, y_pred))
    # Evaluate the models using crossvalidation
    if(len(X_train)>1):
        scores = cross_val_score(regressor, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=min(10,len(X_train)))
    else:
        scores = np.array([0])
        
    #print("MSE = {:.2e}(+/- {:.2e})".format(-scores.mean(), scores.std()))
    return regressor,rmse_score,-scores.mean(),scores.std(), rscore


In [8]:
# Decision Tree

def DecisionTreeClassification_Regressor(train_df):
    # Importing the dataset
    dataset = train_df.copy()
    #print(len(dataset))
    
    dataset['UnitPrice'] = dataset['UnitPrice'].astype(str)
    X_train = dataset.iloc[:, :-1].values
    y_train = dataset.iloc[:, -1].values

    # Training the Decision Tree Regression model on the Training set
    # regressor = DecisionTreeRegressor(max_depth = 9, random_state = 0)
    #regressor = DecisionTreeRegressor(min_samples_leaf = 2, max_depth = 10, random_state = 0)
    regressor = DecisionTreeClassifier(max_depth = 7, random_state = 42)
    #regressor = DecisionTreeRegressor(max_depth = 7, random_state = 42)
    regressor.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = regressor.predict(X_train)
    #print(np.concatenate((y_pred.reshape(len(y_pred),1), y_train.reshape(len(y_train),1)),1))

    # Evaluating the Model Performance
    rscore = r2_score(y_train, y_pred)
    rmse_score = np.sqrt(metrics.mean_squared_error(y_train, y_pred))
    len_uniq = len(dataset['UnitPrice'].unique())
    
    # Evaluate the models using crossvalidation
    #if(len(X_train)>1) and len_uniq>1 and len(X_train)> len_uniq:
    #    scores = cross_val_score(regressor, X_train, y_train,
    #                         scoring="neg_mean_squared_error", cv=min(10,len(X_train)))
    #else:
    scores = np.array([0])
        
    #print("MSE = {:.2e}(+/- {:.2e})".format(-scores.mean(), scores.std()))
    return regressor,rmse_score,-scores.mean(),scores.std(), rscore, y_pred


In [9]:
# Decision Tree

def DecisionTreeRegression_Regressor(train_df):
    # Importing the dataset
    dataset = train_df.copy()
    #print(len(dataset))

    X_train = dataset.iloc[:, :-1].values
    y_train = dataset.iloc[:, -1].values

    # Training the Decision Tree Regression model on the Training set
    # regressor = DecisionTreeRegressor(max_depth = 9, random_state = 0)
    #regressor = DecisionTreeRegressor(min_samples_leaf = 2, max_depth = 10, random_state = 0)
    #regressor = DecisionTreeClassifier()
    regressor = DecisionTreeRegressor(max_depth = 7, random_state = 42)
    
    
    regressor.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = regressor.predict(X_train)
    #print(np.concatenate((y_pred.reshape(len(y_pred),1), y_train.reshape(len(y_train),1)),1))

    # Evaluating the Model Performance
    rscore = r2_score(y_train, y_pred)
    rmse_score = np.sqrt(metrics.mean_squared_error(y_train, y_pred))
     # Evaluate the models using crossvalidation
    if(len(X_train)>1):
        scores = cross_val_score(regressor, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=min(10,len(X_train)))
    else:
        scores = np.array([0])
        
    #print("MSE = {:.2e}(+/- {:.2e})".format(-scores.mean(), scores.std()))
    return regressor,rmse_score,-scores.mean(),scores.std(), rscore, y_pred


In [10]:
# SVR Regressor

parameters = [{'C':[0.25, 0.5, 0.75, 1], 'kernel':['rbf'], 'gamma':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],\
              'epsilon':[0.1, 0.3 , 0.5]}]

def SVR_Regressor(train_df):
    # Importing the dataset
    dataset = train_df.copy()
    #print(len(dataset))

    X_train = dataset.iloc[:, :-1].values
    y_train = dataset.iloc[:, -1].values
    y_train = y_train.reshape(len(y_train),1)

    # Feature Scaling
    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X_train = sc_X.fit_transform(X_train)
    y_train = sc_y.fit_transform(y_train)
    
    """if(len(X_train)>1):
        # Gridsearch to get best regressor
        grid_search = GridSearchCV(estimator = SVR(), param_grid = parameters, scoring = 'neg_mean_squared_error',\
                                  cv = min(10,len(X_train)), n_jobs = -1)
        grid_search.fit(X_train, y_train)
        best_sc = grid_search.best_score_
        best_param = grid_search.best_params_
        regressor = SVR(C = best_param['C'], epsilon = best_param['epsilon'], kernel = best_param['kernel'],\
                        gamma = best_param['gamma'])
        
    else:"""
    best_sc = ''
    best_param = ''
    #regressor = SVR()
    
    # Training the SVR model on the Training set    
    #regressor = SVR(C=1, epsilon=0.1, kernel = 'rbf')
    regressor = SVR(C=0.5, epsilon=0.3, kernel = 'linear')
    regressor.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = sc_y.inverse_transform(regressor.predict(sc_X.transform(X_train)))
    #print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

    # Evaluating the Model Performance
    rscore = r2_score(sc_y.inverse_transform(y_train), y_pred)
    rmse_score = np.sqrt(metrics.mean_squared_error(y_train, y_pred))
    
    # Evaluate the models using crossvalidation
    if(len(X_train)>1):
        scores = cross_val_score(regressor, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=min(10,len(X_train)))
    else:
        scores = np.array([0])
    
    #print(best_sc)
    #print(best_param)
    
    #print("MSE = {:.2e}(+/- {:.2e})".format(-scores.mean(), scores.std()))
    return regressor,rmse_score,-scores.mean(),scores.std(), rscore, best_sc, best_param, sc_X, sc_y


In [11]:
# Random Forest

def RandomForestRegression_Regressor(train_df):
    # Importing the dataset
    dataset = train_df.copy()
    #print(len(dataset))

    X_train = dataset.iloc[:, :-1].values
    y_train = dataset.iloc[:, -1].values

    # Training the Decision Tree Regression model on the Training set
    regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
    regressor.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = regressor.predict(X_train)
    #print(np.concatenate((y_pred.reshape(len(y_pred),1), y_train.reshape(len(y_train),1)),1))

    # Evaluating the Model Performance
    rscore = r2_score(y_train, y_pred)
    #print(rscore)
    rmse_score = np.sqrt(metrics.mean_squared_error(y_train, y_pred))
     # Evaluate the models using crossvalidation
    if(len(X_train)>1):
        scores = cross_val_score(regressor, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=min(10,len(X_train)))
    else:
        scores = np.array([0])
        
    #print("MSE = {:.2e}(+/- {:.2e})".format(-scores.mean(), scores.std()))
    return regressor,rmse_score,-scores.mean(),scores.std(), rscore, y_pred

In [12]:
# XG Boost

def XGB_Regressor(train_df):
    # Importing the dataset
    dataset = train_df.copy()
    #print(len(dataset))

    X_train = dataset.iloc[:, :-1].values
    y_train = dataset.iloc[:, -1].values

    # Training the Decision Tree Regression model on the Training set
    #regressor = XGBRegressor(max_depth = 15, n_estimators=1000)
    regressor = XGBRegressor()
    #eval_set = [(X_train, y_train)]
    #eval_metric = ["rmse"]
    regressor.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = regressor.predict(X_train)
    #print(np.concatenate((y_pred.reshape(len(y_pred),1), y_train.reshape(len(y_train),1)),1))

    # Evaluating the Model Performance
    rscore = r2_score(y_train, y_pred)
    #print(rscore)
    rmse_score = np.sqrt(metrics.mean_squared_error(y_train, y_pred))
    # Evaluate the models using crossvalidation
    if(len(X_train)>1):
        scores = cross_val_score(regressor, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=min(10,len(X_train)))
    else:
        scores = np.array([0])
        
    #print("MSE = {:.2e}(+/- {:.2e})".format(-scores.mean(), scores.std()))
    return regressor,rmse_score,-scores.mean(),scores.std(), rscore, y_pred


In [13]:
# Predict Function

def predict_function(regres_model, X_test):
    # Predicting the Test set results
    y_pred = regres_model.predict(X_test)
    return y_pred

In [14]:
# Predicting for the rows for which there is no description available

final_df = pd.DataFrame()

test_data_unique_desc = test_data_unique_desc[['CustomerID', 'Quantity', 'Country35', 'days_since_first_sold']]
test_data_unique_desc = test_data_unique_desc.drop_duplicates()
test_data_unique_desc.to_csv('test_data_unique_desc.csv', index = False)

for cust_id, split_df in test_data_unique_desc.groupby('CustomerID'):
    #print(cust_id)
    
    # Getting All data for that customerid from train data
    #print('length before selecting customerid',len(train_df_NoDesc))
    
    # Just Trying
    train_df_CountQuantDays_x = train_df_NoDesc[train_df_NoDesc['CustomerID'] == cust_id]
    
    #train_df_CountQuantDays_x = train_df_NoDesc.copy()
    train_df_CountQuantDays_x.drop('CustomerID', axis = 1, inplace = True)
    
    #print('length after selecting customerid',len(train_df_CountQuantDays_x))
    
    # Removing CustomerID and aggregating
    train_df_CountQuantDays_x = train_df_CountQuantDays_x.groupby(['days_since_first_sold', 'Country35','Quantity'], as_index = False).\
                                agg({'UnitPrice':'mean'})
    
    split_df = split_df[['days_since_first_sold', 'Country35','Quantity']]
    split_df.reset_index(drop=True, inplace = True)
    
    # Drop duplicate and then Group by to get avg price in training data
    # Fitting the model using a generic Function
    #print('Training on columns',train_df_CountQuantDays_x.columns)
    regressor, rscore_mlr, cv_mean, cv_sd, rscore = MultipleLinearRegression_Regressor(train_df_CountQuantDays_x)
    X_test = split_df.iloc[:, :].values
    y_pred = predict_function(regressor,X_test)
    #print('Testing on columns',split_df.columns)
    
    split_df['y_pred'] = y_pred
    
    #overwriting by avg price
    #split_df['y_pred'] = float(train_df_CountQuantDays_x['AvgPrice'].mean())
    #print('Mean Value for the country', train_df_CountQuantDays_x['AvgPrice'].mean())
    
    split_df['CustomerID'] = cust_id
    final_df = final_df.append(split_df)
    #print("-------------------------------------------------------------------------------------------")
    
final_df.to_csv('final_df_test_custid.csv', index = False)
    
    

In [15]:
test_data_rep_desc.head()

,CustomerID,Description,Quantity,Country35,days_since_first_sold,Price_Mode_Sum,Price_MeanDesc,Price_MedianDesc,Price_MaxDesc,Price_MinDesc,Price_StDevDesc,Price_Mean_Customer,Price_StDev_Customer,Quantity_Mean,Quantity_Median,Quantity_Max,Quantity_Min,Quantity_StDev,Price_Mode1
0,16161,1079,3,1,72,3.30,1.650000,1.65,1.65,1.65,0.000000,3.364909,3.475940,7.684211,6.0,48,1,9.389774,1.65
1,17341,3457,1,1,6,2.08,1.193421,1.25,2.46,0.83,0.272806,2.082536,1.563595,5.527273,3.0,24,1,5.776243,1.25
2,15158,694,36,1,328,7.70,3.871658,4.25,7.46,2.95,0.489634,1.820000,1.376554,5.258389,4.0,36,1,7.243505,4.25
3,16033,3473,2,1,206,3.10,1.642658,1.65,3.29,1.25,0.261951,3.418571,2.832470,11.314779,12.0,192,1,15.310779,1.65
4,15351,871,1,1,339,23.70,12.654669,12.75,24.96,8.12,0.863610,2.833780,2.262824,3.704104,2.0,160,1,10.071870,12.75


In [16]:
# Predicting for the rows for which there is customer in special list

final_df_cust = pd.DataFrame()

test_data_rep_desc1 = test_data_rep_desc[['CustomerID', 'Description', 'Quantity', 'Country35',\
                                          'Price_Mode_Sum','days_since_first_sold'] + var_list]
test_data_rep_desc1 = test_data_rep_desc1.drop_duplicates()
#test_data_rep_desc.to_csv('test_data_rep_desc.csv', index = False)
test_data_rep_desc1['key'] = range(len(test_data_rep_desc1))

#cust_uniq = train_df_customers['CustomerID'].unique()

train_df_customers_unique = train_df_customers[['CustomerID', 'Description']].drop_duplicates()

test_data_rep_desc_cust = train_df_customers_unique.merge(test_data_rep_desc1, on =['CustomerID', 'Description'],\
                                                          how = 'inner')

test_data_rep_desc_norm = test_data_rep_desc1[~test_data_rep_desc1['key'].isin(test_data_rep_desc_cust['key'])]

test_data_rep_desc_cust.drop('key', axis = 1, inplace = True)
test_data_rep_desc_norm.drop('key', axis = 1, inplace = True)

test_data_rep_desc_cust.drop('Price_Mode_Sum', axis =1, inplace = True)
test_data_rep_desc_cust.drop(var_list, axis =1, inplace = True)
#test_data_rep_desc_cust = test_data_rep_desc[test_data_rep_desc['CustomerID'].isin(cust_uniq)]
#test_data_rep_desc_norm = test_data_rep_desc[~test_data_rep_desc['CustomerID'].isin(cust_uniq)]
print(len(test_data_rep_desc_cust))
print(len(test_data_rep_desc_norm))

for key, split_df in test_data_rep_desc_cust.groupby(['CustomerID', 'Description']):
    cust_id = key[0]
    desc_id = key[1]
    print(cust_id, desc_id)
    
    # Getting All data for that customerid from train data
    #print('length before selecting customerid',len(train_df_customers))
    
    # Filtering
    train_df_x = train_df_customers[(train_df_customers['CustomerID'] == cust_id) & \
                                    (train_df_customers['Description'] == desc_id)]
    
    
    train_df_x.drop('CustomerID', axis = 1, inplace = True)
    train_df_x.drop('Description', axis = 1, inplace = True)
    
    #print('length after selecting customerid',len(train_df_CountQuantDays_x))
    
    # Removing CustomerID and aggregating
    train_df_x = train_df_x.groupby(['days_since_first_sold', 'Country35','Quantity'], as_index = False).\
                                agg({'UnitPrice':'mean'})
    
    split_df = split_df[['days_since_first_sold', 'Country35','Quantity']]
    split_df.reset_index(drop=True, inplace = True)
    
    # Drop duplicate and then Group by to get avg price in training data
    # Fitting the model using a generic Function
    #print('Training on columns',train_df_CountQuantDays_x.columns)
    #regressor, rscore_mlr, cv_mean, cv_sd, rscore = MultipleLinearRegression_Regressor(train_df_CountQuantDays_x)
    regressor, rscore_mlr, cv_mean, cv_sd, rscore, y_pred_dt_x = DecisionTreeRegression_Regressor(train_df_x)
    
    X_test = split_df.iloc[:, :].values
    y_pred = predict_function(regressor,X_test)
    #print('Testing on columns',split_df.columns)
    
    split_df['y_pred'] = y_pred
    
    #overwriting by avg price
    #split_df['y_pred'] = float(train_df_CountQuantDays_x['AvgPrice'].mean())
    #print('Mean Value for the country', train_df_CountQuantDays_x['AvgPrice'].mean())
    
    split_df['CustomerID'] = cust_id
    split_df['Description'] = desc_id
    final_df_cust = final_df_cust.append(split_df)
    #print("-------------------------------------------------------------------------------------------")
    #break
    
final_df_cust.to_csv('final_df_test_custid.csv', index = False)
    

1835
119283
14088 30
14088 55
14088 184
14088 193
14088 205
14088 253
14088 271
14088 288
14088 528
14088 529
14088 536
14088 655
14088 693
14088 725
14088 738
14088 739
14088 901
14088 1015
14088 1020
14088 1022
14088 1023
14088 1033
14088 1041
14088 1132
14088 1184
14088 1203
14088 1205
14088 1286
14088 1287
14088 1320
14088 1380
14088 1381
14088 1387
14088 1443
14088 1453
14088 1487
14088 1518
14088 1535
14088 1542
14088 1658
14088 1665
14088 1770
14088 1839
14088 1870
14088 2158
14088 2328
14088 2352
14088 2403
14088 2478
14088 2539
14088 2708
14088 2741
14088 2776
14088 2780
14088 2790
14088 2934
14088 2941
14088 3006
14088 3012
14088 3124
14088 3369
14088 3398
14088 3411
14088 3424
14088 3504
14088 3553
14088 3578
14088 3580
14088 3581
14088 3696
14088 3764
14088 3772
14088 3782
14088 3883
14096 12
14096 19
14096 45
14096 48
14096 52
14096 54
14096 58
14096 59
14096 63
14096 69
14096 74
14096 75
14096 86
14096 101
14096 102
14096 103
14096 104
14096 105
14096 106
14096 109
14096 

14298 1221
14298 1224
14298 1235
14298 1236
14298 1275
14298 1342
14298 1361
14298 1396
14298 1402
14298 1427
14298 1430
14298 1444
14298 1480
14298 1481
14298 1518
14298 1535
14298 1542
14298 1544
14298 1551
14298 1553
14298 1554
14298 1577
14298 1578
14298 1579
14298 1581
14298 1585
14298 1602
14298 1607
14298 1609
14298 1610
14298 1613
14298 1614
14298 1617
14298 1618
14298 1649
14298 1666
14298 1669
14298 1679
14298 1691
14298 1692
14298 1701
14298 1736
14298 1741
14298 1754
14298 1759
14298 1760
14298 1761
14298 1763
14298 1765
14298 1766
14298 1769
14298 1770
14298 1772
14298 1774
14298 1775
14298 1776
14298 1781
14298 1783
14298 1784
14298 1813
14298 1821
14298 1838
14298 1839
14298 1855
14298 1874
14298 1875
14298 1879
14298 1936
14298 1938
14298 1939
14298 1943
14298 1973
14298 2024
14298 2041
14298 2058
14298 2110
14298 2111
14298 2113
14298 2117
14298 2119
14298 2129
14298 2133
14298 2139
14298 2142
14298 2143
14298 2145
14298 2146
14298 2165
14298 2166
14298 2203
14298 2265

In [ ]:
# For BAD Description

#train_df_in = 
#test_df_in = 

In [ ]:
# Predicting for the rows for which there is a description available in training data and Evaluating all Models

final_df_desc = pd.DataFrame()

#test_data_rep_desc_norm = test_data_rep_desc_norm[['Description', 'Quantity', 'Country35','days_since_first_sold','Price_Mode_Sum']]
#test_data_rep_desc_norm = test_data_rep_desc_norm.drop_duplicates()
test_data_rep_desc_norm1 = test_data_rep_desc_norm[[ 'Price_Mode_Sum','Quantity', 'Country35','days_since_first_sold']\
                                                  +var_list ]

train_df_normal = pd.DataFrame()

i =0

score_df = pd.DataFrame()

complete_list = [ 'Quantity','Country35', 'days_since_first_sold']+var_list
target_col = 'UnitPrice'

for description_id, split_df in test_data_rep_desc_norm1.groupby('Price_Mode_Sum'):
    #print('-------------------------',description_id, '---------------------------------')
    # Getting All data for that country from train data
    train_df_x = train_df[train_df['Price_Mode_Sum'] == description_id]
    count_descriptions = train_df_x['Description'].nunique()
    
    train_df_x = train_df_x.sort_values(['days_since_first_sold'], ascending = True)
    train_df_x.index = range(len(train_df_x))
    
    train_df_x2 = train_df_x.copy()
    
    # Changing Column Orders
    split_df = split_df[complete_list]
    train_df_x = train_df_x[complete_list+[target_col]]
    split_df.index = range(len(split_df))
    
    
    val1 = train_df_x.loc[0,'UnitPrice']
    val2 = train_df_x.loc[len(train_df_x)-1,'UnitPrice']
   
    X_test = split_df.iloc[:, :].values
        
    #print('Training on columns',train_df_x.columns)
    #print('Testing on columns',split_df.columns)
    
    # Fitting the model using a generic Function
    
    #Multiple Linear Regression Model
    #regressor_mlr, rmse_mlr, cv_mean_mlr, cv_sd_mlr, rscore_mlr = MultipleLinearRegression_Regressor( train_df_x )
    
    #Decision Tree Model
    regressor_dt, rmse_dt, cv_mean_dt, cv_sd_dt, rscore_dt, y_pred_dt_x = DecisionTreeRegression_Regressor( train_df_x )
    
    #Decision Tree Model Classification
    regressor_dt_clf, rmse_dt_clf, cv_mean_dt_clf, cv_sd_dt_clf, rscore_dt_clf, y_pred_dt_clf_x = DecisionTreeClassification_Regressor( train_df_x )
    
    #SVR Model
    #Feature Scaling
    #regressor_svr, rmse_svr, cv_mean_svr, cv_sd_svr, rscore_svr, best_sc, best_param, sc_X, sc_y = SVR_Regressor\
    #                                                                               ( train_df_x ) 
    
    #Random Forest Model
    regressor_rf, rmse_rf, cv_mean_rf, cv_sd_rf, rscore_rf, y_pred_rf = RandomForestRegression_Regressor( train_df_x )
    
    #XGB_Regressor
    regressor_xgb, rmse_xgb, cv_mean_xgb, cv_sd_xgb, rscore_xgb, y_pred_xgb = XGB_Regressor( train_df_x )
    
    #print(X_test)
    
    #if cv_mean_svr+cv_sd_svr < cv_mean_dt+cv_sd_dt:
    len_uniq = len(train_df_x['UnitPrice'].unique())
    
    # If error Condition
    y_pred = predict_function(regressor_dt, X_test)
    
    
    #split_df['y_mlr'] = predict_function(regressor_mlr, X_test)
    split_df['y_dt'] = predict_function(regressor_dt, X_test)
    split_df['y_dt_clf'] = predict_function(regressor_dt_clf, X_test)
    #split_df['y_svr'] = sc_y.inverse_transform(regressor_svr.predict(sc_X.transform(X_test)))
    split_df['y_rf'] = predict_function(regressor_rf, X_test)
    split_df['y_xgb'] = predict_function(regressor_xgb, X_test)
    
    # Checking if there is only one value in the unique price
    
    if len_uniq == 1:
        split_df['y_pred'] = list(train_df_x['UnitPrice'].unique())[0]
        
    else:
        split_df['y_pred'] = y_pred
    
    # Adding Description combining to the overall dataset
    split_df['Price_Mode_Sum'] = description_id
    final_df_desc = final_df_desc.append(split_df)
    if i%100 == 0:
        print(description_id)
    
    score_df_temp = pd.DataFrame({'Description':  [description_id], \
                                  #'rmse_mlr':[rmse_mlr],\
                                  'rmse_dt': [rmse_dt],\
                                  'rmse_dt_clf': [rmse_dt_clf],\
                                  #'rmse_svr':[rmse_svr],\
                                  'rmse_rf':[rmse_rf],\
                                  'rmse_xgb':[rmse_xgb],\
                                  #'rscore_mlr':[rscore_mlr],\
                                  'rscore_dt':[rscore_dt],\
                                  'rscore_dt_clf':[rscore_dt_clf],\
                                  #'rscore_svr':[rscore_svr],\
                                  'rscore_rf':[rscore_rf],\
                                  'rscore_xgb':[rscore_xgb],\
                                  'depth_dt':[regressor_dt.tree_.max_depth],\
                                  'depth_dt_clf':[regressor_dt_clf.tree_.max_depth],\
                                  #'depth_rf':[regressor_rf.tree_.max_depth],\
                                  #'depth_xgb':[regressor_xgb.tree_.max_depth],\
                                  #'cv_mean_mlr':[cv_mean_mlr],\
                                  'cv_mean_dt':[cv_mean_dt],\
                                  'cv_mean_dt_clf':[cv_mean_dt_clf],\
                                  #'cv_mean_svr':[cv_mean_svr],\
                                  'cv_mean_rf':[cv_mean_rf],\
                                  'cv_mean_xgb':[cv_mean_xgb],\
                                  #'cv_sd_mlr':[cv_sd_mlr],\
                                  'cv_sd_dt':[cv_sd_dt],\
                                  'cv_sd_dt_clf':[cv_sd_dt_clf],\
                                  #'cv_sd_svr':[cv_sd_svr],\
                                  #'cv_sd_rf':[cv_sd_rf],\
                                  #'cv_sd_xgb':[cv_sd_xgb],\
                                  #'svr_best_sc':[best_sc], 'svr_best_param':[best_param],\
                                  'length_of_train': [len(train_df_x)],\
                                  'unique_length':[len_uniq],\
                                  'length_of_test':[len(split_df)],\
                                  'first_last_same':[(val1 == val2)],\
                                  'count_descriptions':count_descriptions,\
                                  'length_available_in_org_test':[len(test_df_org[test_df_org['Description'] == description_id])  ]})
    
    score_df = score_df.append(score_df_temp)
    i += 1
    
    #train_df_x2['Description'] = description_id
    train_df_x2['y_pred_dt'] = y_pred_dt_x
    train_df_x2['y_pred_dt_clf'] = y_pred_dt_clf_x
    train_df_x2['y_pred_rf'] = y_pred_rf
    train_df_x2['y_pred_xgb'] = y_pred_xgb
    train_df_normal = train_df_normal.append(train_df_x2)
    #if i>10:
    #    break
    
final_df_desc.to_csv('final_df_test_desc.csv', index = False)
score_df.to_csv('score_df_desc.csv', index = False)



In [ ]:
# Reading Description Var 

In [17]:
# Predicting for the rows for which there is a description available in training data and Evaluating all Models

final_df_desc2 = pd.DataFrame()
var_list2 = []

#test_data_rep_desc_norm = test_data_rep_desc_norm[['Description', 'Quantity', 'Country35','days_since_first_sold','Price_Mode_Sum']]
#test_data_rep_desc_norm = test_data_rep_desc_norm.drop_duplicates()
test_data_rep_desc_norm2 = test_data_rep_desc_norm[[ 'Description','Quantity', 'Country35','days_since_first_sold']\
                                                  +var_list2 ]

train_df_normal2 = pd.DataFrame()

i =0

score_df2 = pd.DataFrame()


complete_list = [ 'Quantity','Country35', 'days_since_first_sold']+var_list2
target_col = 'UnitPrice'

for description_id, split_df in test_data_rep_desc_norm2.groupby('Description'):
    #print('-------------------------',description_id, '---------------------------------')
    # Getting All data for that country from train data
    train_df_x = train_df[train_df['Description'] == description_id]
    
    train_df_x = train_df_x.sort_values(['days_since_first_sold'], ascending = True)
    train_df_x.index = range(len(train_df_x))
    
    train_df_x2 = train_df_x.copy()
    
    # Changing Column Orders
    split_df = split_df[complete_list]
    train_df_x = train_df_x[complete_list+[target_col]]
    split_df.index = range(len(split_df))
    
    
    val1 = train_df_x.loc[0,'UnitPrice']
    val2 = train_df_x.loc[len(train_df_x)-1,'UnitPrice']
   
    X_test = split_df.iloc[:, :].values
        
    #print('Training on columns',train_df_x.columns)
    #print('Testing on columns',split_df.columns)
    
    # Fitting the model using a generic Function
    
    #Multiple Linear Regression Model
    #regressor_mlr, rmse_mlr, cv_mean_mlr, cv_sd_mlr, rscore_mlr = MultipleLinearRegression_Regressor( train_df_x )
    
    #Decision Tree Model
    regressor_dt, rmse_dt, cv_mean_dt, cv_sd_dt, rscore_dt, y_pred_dt_x = DecisionTreeRegression_Regressor( train_df_x )
    
    #Decision Tree Model Classification
    regressor_dt_clf, rmse_dt_clf, cv_mean_dt_clf, cv_sd_dt_clf, rscore_dt_clf, y_pred_dt_clf_x = DecisionTreeClassification_Regressor( train_df_x )
    
    #SVR Model
    #Feature Scaling
    #regressor_svr, rmse_svr, cv_mean_svr, cv_sd_svr, rscore_svr, best_sc, best_param, sc_X, sc_y = SVR_Regressor\
    #                                                                               ( train_df_x ) 
    
    #Random Forest Model
    regressor_rf, rmse_rf, cv_mean_rf, cv_sd_rf, rscore_rf, y_pred_rf = RandomForestRegression_Regressor( train_df_x )
    
    #XGB_Regressor
    regressor_xgb, rmse_xgb, cv_mean_xgb, cv_sd_xgb, rscore_xgb, y_pred_xgb = XGB_Regressor( train_df_x )
    
    #print(X_test)
    
    #if cv_mean_svr+cv_sd_svr < cv_mean_dt+cv_sd_dt:
    len_uniq = len(train_df_x['UnitPrice'].unique())
    
    # If error Condition
    if len_uniq==2:
         y_pred = predict_function(regressor_dt_clf, X_test)
    else:
        y_pred = predict_function(regressor_dt, X_test)
    
    
    #split_df['y_mlr'] = predict_function(regressor_mlr, X_test)
    split_df['y_dt'] = predict_function(regressor_dt, X_test)
    split_df['y_dt_clf'] = predict_function(regressor_dt_clf, X_test)
    #split_df['y_svr'] = sc_y.inverse_transform(regressor_svr.predict(sc_X.transform(X_test)))
    split_df['y_rf'] = predict_function(regressor_rf, X_test)
    split_df['y_xgb'] = predict_function(regressor_xgb, X_test)
    
    # Checking if there is only one value in the unique price
    
    if len_uniq == 1:
        split_df['y_pred'] = list(train_df_x['UnitPrice'].unique())[0]
        
    else:
        split_df['y_pred'] = y_pred
    
    # Adding Description combining to the overall dataset
    split_df['Description'] = description_id
    final_df_desc2 = final_df_desc2.append(split_df)
    if i%100 == 0:
        print(description_id)
        #print('R-score', rscore_dt)
        #print("-------------------------------------------------------------------------------------------")
    
    #print(description_id)
    #print("-------------------------------------------------------------------------------------------")
    
    score_df_temp = pd.DataFrame({'Description':  [description_id], \
                                  #'rmse_mlr':[rmse_mlr],\
                                  'rmse_dt': [rmse_dt],\
                                  'rmse_dt_clf': [rmse_dt_clf],\
                                  #'rmse_svr':[rmse_svr],\
                                  'rmse_rf':[rmse_rf],\
                                  'rmse_xgb':[rmse_xgb],\
                                  #'rscore_mlr':[rscore_mlr],\
                                  'rscore_dt':[rscore_dt],\
                                  'rscore_dt_clf':[rscore_dt_clf],\
                                  #'rscore_svr':[rscore_svr],\
                                  'rscore_rf':[rscore_rf],\
                                  'rscore_xgb':[rscore_xgb],\
                                  'depth_dt':[regressor_dt.tree_.max_depth],\
                                  'depth_dt_clf':[regressor_dt_clf.tree_.max_depth],\
                                  #'depth_rf':[regressor_rf.tree_.max_depth],\
                                  #'depth_xgb':[regressor_xgb.tree_.max_depth],\
                                  #'cv_mean_mlr':[cv_mean_mlr],\
                                  'cv_mean_dt':[cv_mean_dt],\
                                  'cv_mean_dt_clf':[cv_mean_dt_clf],\
                                  #'cv_mean_svr':[cv_mean_svr],\
                                  'cv_mean_rf':[cv_mean_rf],\
                                  'cv_mean_xgb':[cv_mean_xgb],\
                                  #'cv_sd_mlr':[cv_sd_mlr],\
                                  'cv_sd_dt':[cv_sd_dt],\
                                  'cv_sd_dt_clf':[cv_sd_dt_clf],\
                                  #'cv_sd_svr':[cv_sd_svr],\
                                  #'cv_sd_rf':[cv_sd_rf],\
                                  #'cv_sd_xgb':[cv_sd_xgb],\
                                  #'svr_best_sc':[best_sc], 'svr_best_param':[best_param],\
                                  'length_of_train': [len(train_df_x)],\
                                  'unique_length':[len_uniq],\
                                  'length_of_test':[len(split_df)],\
                                  'first_last_same':[(val1 == val2)],\
                                  #'count_descriptions':count_descriptions,\
                                  'DataAvailable':[train_df_x['days_since_first_sold'].max()],\
                                  'length_available_in_org_test':[len(test_df_org[test_df_org['Description'] == description_id])  ]})
    
    score_df2 = score_df2.append(score_df_temp)
    i += 1
    
    #train_df_x2['Description'] = description_id
    train_df_x2['y_pred_dt'] = y_pred_dt_x
    train_df_x2['y_pred_dt_clf'] = y_pred_dt_clf_x
    train_df_x2['y_pred_rf'] = y_pred_rf
    train_df_x2['y_pred_xgb'] = y_pred_xgb
    train_df_normal2 = train_df_normal2.append(train_df_x2)
    #if i>100:
    #    break
    
final_df_desc2.to_csv('final_df_test_desc2.csv', index = False)
score_df2.to_csv('score_df_desc2.csv', index = False)



0
109
223
335
458
568
677
781
902
1021
1131
1245
1360
1486
1592
1701
1809
1929
2035
2143
2263
2368
2493
2613
2731
2833
2949
3053
3157
3267
3371
3479
3590
3706
3823


In [ ]:
train_df_normal.head()

In [ ]:
#train_df_normal.to_csv('Train_with_values.csv', index = False)

print('-----------------------RMSE ALL-----------------------')
print('DT', np.sqrt(metrics.mean_squared_error(train_df_normal['UnitPrice'], train_df_normal['y_pred_dt'])))
print('DT CLF', np.sqrt(metrics.mean_squared_error(train_df_normal['UnitPrice'], train_df_normal['y_pred_dt_clf'])))
print('RF', np.sqrt(metrics.mean_squared_error(train_df_normal['UnitPrice'], train_df_normal['y_pred_rf'])))
print('XGB', np.sqrt(metrics.mean_squared_error(train_df_normal['UnitPrice'], train_df_normal['y_pred_xgb'])))

bad_desc_list = [2140, 2624]
train_df_normal_wo_bad = train_df_normal[~(train_df_normal['Description'].isin(bad_desc_list))]
train_df_normal_bad = train_df_normal[(train_df_normal['Description'].isin(bad_desc_list))]

print('-----------------------RMSE without bad descriptions---------------------------')
print('DT', np.sqrt(metrics.mean_squared_error(train_df_normal_wo_bad['UnitPrice'], train_df_normal_wo_bad['y_pred_dt'])))
print('DT CLF', np.sqrt(metrics.mean_squared_error(train_df_normal_wo_bad['UnitPrice'], train_df_normal_wo_bad['y_pred_dt_clf'])))
print('RF', np.sqrt(metrics.mean_squared_error(train_df_normal_wo_bad['UnitPrice'], train_df_normal_wo_bad['y_pred_rf'])))
print('XGB', np.sqrt(metrics.mean_squared_error(train_df_normal_wo_bad['UnitPrice'], train_df_normal_wo_bad['y_pred_xgb'])))

print('-----------------------RMSE of bad descriptions-----------------------')
print('DT', np.sqrt(metrics.mean_squared_error(train_df_normal_bad['UnitPrice'], train_df_normal_bad['y_pred_dt'])))
print('DT CLF', np.sqrt(metrics.mean_squared_error(train_df_normal_bad['UnitPrice'], train_df_normal_bad['y_pred_dt_clf'])))
print('RF', np.sqrt(metrics.mean_squared_error(train_df_normal_bad['UnitPrice'], train_df_normal_bad['y_pred_rf'])))
print('XGB', np.sqrt(metrics.mean_squared_error(train_df_normal_bad['UnitPrice'], train_df_normal_bad['y_pred_xgb'])))

#print('Combined', np.sqrt(metrics.mean_squared_error(train_df_normal['UnitPrice'], train_df_normal['y_dt_comb'])))

var_list

In [89]:
#train_df_normal.to_csv('Train_with_values.csv', index = False)

print('-----------------------RMSE ALL-----------------------')
print('DT', np.sqrt(metrics.mean_squared_error(train_df_normal2['UnitPrice'], train_df_normal2['y_pred_dt'])))
print('DT CLF', np.sqrt(metrics.mean_squared_error(train_df_normal2['UnitPrice'], train_df_normal2['y_pred_dt_clf'])))
print('RF', np.sqrt(metrics.mean_squared_error(train_df_normal2['UnitPrice'], train_df_normal2['y_pred_rf'])))
print('XGB', np.sqrt(metrics.mean_squared_error(train_df_normal2['UnitPrice'], train_df_normal2['y_pred_xgb'])))
#print('Comb', np.sqrt(metrics.mean_squared_error(train_df_normal2['UnitPrice'], train_df_normal2['y_pred'])))

bad_desc_list = [2624]
train_df_normal_wo_bad = train_df_normal2[~(train_df_normal2['Description'].isin(bad_desc_list))]
train_df_normal_bad = train_df_normal2[(train_df_normal2['Description'].isin(bad_desc_list))]

print('-----------------------RMSE without bad descriptions---------------------------')
print('DT', np.sqrt(metrics.mean_squared_error(train_df_normal_wo_bad['UnitPrice'], train_df_normal_wo_bad['y_pred_dt'])))
print('DT CLF', np.sqrt(metrics.mean_squared_error(train_df_normal_wo_bad['UnitPrice'], train_df_normal_wo_bad['y_pred_dt_clf'])))
print('RF', np.sqrt(metrics.mean_squared_error(train_df_normal_wo_bad['UnitPrice'], train_df_normal_wo_bad['y_pred_rf'])))
print('XGB', np.sqrt(metrics.mean_squared_error(train_df_normal_wo_bad['UnitPrice'], train_df_normal_wo_bad['y_pred_xgb'])))
#print('Comb', np.sqrt(metrics.mean_squared_error(train_df_normal_wo_bad['UnitPrice'], train_df_normal_wo_bad['y_pred'])))

print('-----------------------RMSE of bad descriptions-----------------------')
print('DT', np.sqrt(metrics.mean_squared_error(train_df_normal_bad['UnitPrice'], train_df_normal_bad['y_pred_dt'])))
print('DT CLF', np.sqrt(metrics.mean_squared_error(train_df_normal_bad['UnitPrice'], train_df_normal_bad['y_pred_dt_clf'])))
print('RF', np.sqrt(metrics.mean_squared_error(train_df_normal_bad['UnitPrice'], train_df_normal_bad['y_pred_rf'])))
print('XGB', np.sqrt(metrics.mean_squared_error(train_df_normal_bad['UnitPrice'], train_df_normal_bad['y_pred_xgb'])))
#print('Comb', np.sqrt(metrics.mean_squared_error(train_df_normal_bad['UnitPrice'], train_df_normal_bad['y_pred'])))

#print('Combined', np.sqrt(metrics.mean_squared_error(train_df_normal['UnitPrice'], train_df_normal['y_dt_comb'])))

var_list

-----------------------RMSE ALL-----------------------
DT 3.8151949607229807
DT CLF 11.648310783904128
RF 4.341698367846394
XGB 3.2703789456808954
-----------------------RMSE without bad descriptions---------------------------
DT 3.687007209297417
DT CLF 11.519146839566362
RF 4.169738620232001
XGB 3.1521330043258695
-----------------------RMSE of bad descriptions-----------------------
DT 18.730404059435568
DT CLF 34.3806573676188
RF 23.0354715223927
XGB 16.620643805430237


[]

In [156]:
#print(final_df_desc.head())
print(final_df_desc2.columns)

#final_df_desc = final_df_desc['Quantity', 'Country35', 'days_since_first_sold','']
final_df_desc_n = final_df_desc2.drop_duplicates()

col_list_to_join = ['Quantity', 'Country35', 'days_since_first_sold', 'Price_MeanDesc',\
       'Price_MedianDesc', 'Price_MaxDesc', 'Price_MinDesc', 'Price_StDevDesc',\
       'Price_Mean_Customer', 'Price_StDev_Customer', 'Quantity_Mean',\
       'Quantity_Median', 'Quantity_Max', 'Quantity_Min', 'Quantity_StDev',\
       'Price_Mode1',\
       'Price_Mode_Sum']

col_list_to_join2 = ['Quantity', 'Country35', 'days_since_first_sold', 'Description']

len(final_df_desc_n)
#len(final_df_desc.drop_duplicates())

Index(['Quantity', 'Country35', 'days_since_first_sold', 'y_dt', 'y_dt_clf',
       'y_rf', 'y_xgb', 'y_pred', 'Description'],
      dtype='object')


113141

In [197]:
# Run at country level for 2624
tr_df_org = pd.read_csv('Train_df_org.csv')
test_df_in = pd.read_csv('Test_df_org.csv', parse_dates = ['InvoiceDate', 'InvoiceDate_org'])

train_df_2624 = tr_df_org[tr_df_org['Description'] == 2624]

print(len(train_df_2624))
train_df_2624 = train_df_2624[train_df_2624['UnitPrice']<100]
print(len(train_df_2624))

train_df_2624.index = range(len(train_df_2624))


test_df_2624 = test_df_in[test_df_in['Description'] == 2624]

#print(test_df_2624.columns)
"""
from sklearn.preprocessing import OneHotEncoder
# creating one hot encoder object 
onehotencoder = OneHotEncoder()
encoder = OneHotEncoder(sparse=False)
train_X_encoded = pd.DataFrame (encoder.fit_transform(train_df_2624[['Country']]))
train_X_encoded.columns = encoder.get_feature_names(['Country'])
train_df_2624 = pd.concat([train_df_2624, train_X_encoded], axis=1)

onehotencoder = OneHotEncoder()
encoder = OneHotEncoder(sparse=False)
test_X_encoded = pd.DataFrame (encoder.fit_transform(test_df_2624[['Country']]))
test_X_encoded.columns = encoder.get_feature_names(['Country'])
test_df_2624 = pd.concat([test_df_2624, test_X_encoded], axis=1)


country_list = list(encoder.get_feature_names(['Country']))
"""
#varlist = ['Quantity_org','days_since_first_sold']+ country_list
varlist = ['Quantity_org','days_since_first_sold']+ ['Country']
print(varlist)
target_col = 'UnitPrice'

X_train = train_df_2624[varlist]
y_train = train_df_2624[target_col]

regressor = DecisionTreeRegressor(max_depth = 7, random_state = 42)
regressor.fit(X_train, y_train)

train_df_2624['y_pred_2624'] = regressor.predict(X_train)
print('2624', np.sqrt(metrics.mean_squared_error(train_df_2624['UnitPrice'], train_df_2624['y_pred_2624'])))

print(len(test_df_2624))
#print(test_df_2624[var_list].head())
test_df_2624['y_2624'] = regressor.predict(test_df_2624[varlist])


#test_df_in = test_df_in.merge(test_df_2624[['Key', 'y_2624']], on = 'Key', how = 'left')

key_df = pd.read_csv('Key_manual.csv')
test_df_in = test_df_in.merge(key_df[['Key', 'org']], on = 'Key', how = 'left').rename(columns = {'org':'y_2624'})

print(len(test_df_in))
#X_train data 

#dt_regressor()


777
769
['Quantity_org', 'days_since_first_sold', 'Country']
2624 1.2438252807288708
356
122049


In [198]:
# Overriding for 2140

# Handling Data For Bad Description 2140 and 2624


test_desc_2140 = pd.read_csv('Manual_Entry.csv', parse_dates = ['InvoiceDate'])

left_on = ['InvoiceNo', 'StockCode', 'Description', 'Quantity_org', 'CustomerID', 'Country']
right_on = ['InvoiceNo', 'StockCode', 'Description', 'Q', 'CustomerID', 'Country']


print(test_df_in[left_on].dtypes)

test_desc_2140 = test_desc_2140.rename(columns = {'Quantity':'Q', 'InvoiceDate':'In'})

print(test_desc_2140[right_on].dtypes)

test_desc_2140 = test_desc_2140[right_on+['Manual_Value']].drop_duplicates()

test_df_org1 = test_df_in.merge(test_desc_2140, left_on = left_on, right_on = right_on, how = 'left')
test_df_org1.drop('UnitPrice', axis = 1 , inplace = True)

print(len(test_df_org1))

test_df_org1.to_csv('Test_Merged.csv', index = False)

InvoiceNo       int64
StockCode       int64
Description     int64
Quantity_org    int64
CustomerID      int64
Country         int64
dtype: object
InvoiceNo      int64
StockCode      int64
Description    int64
Q              int64
CustomerID     int64
Country        int64
dtype: object
122049


In [199]:

#print(test_df_org.head())
#'y_rf':'y_Desc',  y_Generic_desc

#print(test_df_org1.columns)
#print(test_df_org1[['CustomerID','days_since_first_sold', 'Quantity', 'Country35']].dtypes)
#print(final_df[['CustomerID','days_since_first_sold', 'Quantity', 'Country35']].dtypes)



test_df_org1 = test_df_org1.merge(final_df, on = ['CustomerID','days_since_first_sold', 'Quantity', 'Country35'], \
                                 how = 'left').rename(columns = {'y_pred':'y_Generic_cust'})

print(len(test_df_org1))

#print(test_df_org1.columns)
test_df_org2 = test_df_org1.merge(final_df_desc_n, on = col_list_to_join2, \
                                  how = 'left').rename(columns = { \
                                                                  'y_dt':'y_Desc',\
                                                                  'y_pred':'y_Generic_desc'
                                                                 })
#print(test_df_org2.columns)
print(len(test_df_org2))

test_df_org2 = test_df_org2.merge(final_df_cust, on = ['CustomerID','days_since_first_sold', 'Quantity', 'Description',\
                                                       'Country35'], how = 'left').\
                                                        rename(columns = {'y_pred':'y_desc_Cust'})
#print(test_df_org2.columns)
#print(final_df_cust.columns)
print(len(test_df_org2))

"""
tr_df = pd.read_csv('Train_df_org.csv')
train_df2 = tr_df[['days_since_first_sold','CustomerID', 'Quantity', 'Description','Country35','UnitPrice']].\
                                            drop_duplicates(['days_since_first_sold','CustomerID', 'Quantity', 'Description','Country35'])
test_df_org3 = test_df_org2.merge(train_df2, on = ['days_since_first_sold', 'Quantity', 'Description','Country35', 'CustomerID'], how = 'left')\
                                            .rename(columns = {'UnitPrice':'OrgPrice'})
print(len(test_df_org3))
"""

tr_cust_df = pd.read_csv('Customer_Description_Day_Quantity.csv')
train_df2 = tr_cust_df[['days_since_first_sold', 'Quantity', 'Description','CustomerID','Country35','UnitPrice']].drop_duplicates()
test_df_org3 = test_df_org2.merge(train_df2, on = ['days_since_first_sold', 'Quantity', 'Description',\
                                                   'CustomerID','Country35'],\
                                  how = 'left').rename(columns = {'UnitPrice':'OrgPrice'})

print(len(test_df_org3))



122049
122049
122049
122049


In [200]:
#test_df_org3.to_csv('test_df_with_values.csv', index = False)

#print(test_df_org3.dtypes)

In [201]:
test_df_org3['y_final1'] = np.where((test_df_org3["y_Desc"].isnull() == True), test_df_org3["y_Generic_cust"],test_df_org3["y_Desc"] )

test_df_org3['y_final2'] = np.where(test_df_org3["y_desc_Cust"].isnull() == True, test_df_org3["y_final1"],\
                                   test_df_org3["y_desc_Cust"] )

test_df_org3['y_final3'] = np.where(test_df_org3["OrgPrice"].isnull() == True, test_df_org3["y_final2"],\
                                    test_df_org3["OrgPrice"] )

#test_df_org3['y_final4'] = np.where(test_df_org3["Description"].isin([2140]), test_df_org3["Manual_Value"],\
#                                    test_df_org3["y_final3"] )

test_df_org3['y_final5'] = np.where(((test_df_org3["Description"].isin([2624,2140]))&\
                                    ~(test_df_org3["y_2624"].isnull())), test_df_org3["y_2624"],\
                                    test_df_org3["y_final3"] )

test_df_org3['y_final5'] = test_df_org3['y_final5'].apply(float)
test_df_org3['y_final5'] = test_df_org3['y_final5'].apply(abs)
test_df_org3.to_csv('Test_with_y_values.csv', index = False)

In [202]:
# Creating the submission file

test_df_org3['UnitPrice'] = test_df_org3['y_final5']
test_df_org3[['UnitPrice']].to_csv('my_submission_file.csv', index=False)